#Running Our Auto Loader Bronze Layer




Assigning variables with our databricks file share paths that we created previously in 1. 

In [0]:
#%Python # use if you have your default set to another language

source_data_loc = "dbfs:/FileStore/tables/TraderJoesRevenue/Files/"
target_data_loc = "dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue"
checkpoints_loc = "dbfs:/FileStore/tables/TraderJoesRevenue/checkpoints/"
schema_loc = "dbfs:/FileStore/tables/TraderJoesRevenue/schema/"

Reading the cloud files that are stored in our DBFS using structured streaming (Autoloader).

The file watch location can be changed to an Amazon S3 Bucket or Azure FileShare or Blob Storage location

In [0]:
#%Python # use if you have your default set to another language

df=(spark.readStream.format("cloudFiles") \
  .option("cloudFiles.format", "csv") \
  .option("cloudFiles.inferColumnTypes", "true") \
  .option("cloudFiles.schemaLocation", schema_loc) \
  .option("cloudFiles.schemaEvolutionMode", "addNewColumns") \
  .load(source_data_loc) \
  )

df.writeStream \
  .format("delta") \
  .option("mergeSchema", "true") \
  .option("checkpointLocation", checkpoints_loc) \
  .outputMode("append") \
  .trigger(availableNow=True) \
  .start(target_data_loc)